In [ ]:
%matplotlib inline
import rootpy.ROOT as R
R.gROOT.SetBatch(True)
import numpy as np
from rootpy.io import root_open, DoesNotExist
from rootpy.plotting import Canvas, Hist, Legend
import rootpy.plotting.root2matplotlib as rplt
import matplotlib.pyplot as plt

In [ ]:
# 0-5%, 5-10%, 10-20%, 20-30%, 30-40%, 40-60%, 60-80%
labels = ( '0-5%', '5-10%', '10-20%', '20-30%', '30-40%', '40-60%', '60-80%', )

ncoll = {  '19.6' : ( 799.84, 642.84, 458.4, 284.54252, 169.88653, 74.01196, 18.91901 ),
            '11.5' : ( 784.45235, 635.10335, 453.30864, 283.86052, 171.77151, 74.74965, 19.08872),
            '14.5' : ( 787.9, 639, 455, 284, 171, 75, 18.305 )}

def get_evan_spectra( en, plc, c, cl, ch ) :
    ef = root_open( "/Users/danielbrandenburg/bnl/local/work/rcpMaker/script/evan/spectra_" + en + ".root" )
    he = ef.Get( plc + "_" + c + "_" + str(cl) + "_" + str(ch) )
    return he

def to_evan_cen( iCen ) :
    if iCen == 6 :
        return ( 0, 1 )
    if iCen == 0 :
        return (8, 8)

    


def get_evan_rcp( en, plc, c, iCen, iPer, **kwargs ) :
    c1, c2 = to_evan_cen( iCen )
    p1, p2 = to_evan_cen( iPer )
    
    hPer = get_evan_spectra( en, plc, c, p1, p2 )
    hCen = get_evan_spectra( en, plc, c, c1, c2 )
    
    hRatio = hCen.Clone( "even_rcp" + en + plc )
    
    hRatio.Divide( hPer )
    
    nc = ncoll[ en ][ iCen ]
    np = ncoll[ en ][ iPer ]
    
    hRatio.Scale( np / nc )
    if 'color' in kwargs :
        hRatio.SetMarkerColor( kwargs[ 'color' ] )
    return hRatio
    
    

In [5]:
def plot_dans_rcp( plc, charge, iCen, iPer, **kwargs  ) :
    
    
    hRcp = data.Get( "rcp/rcp_" + plc + "_" + charge + "_" + str(iCen) + "_Over_" + str(iPer) )
    hRcp.SetTitle( plc + "_" + charge )
    hRcp.GetYaxis().SetTitle( "(" + labels[iCen] + ") / ( " + labels[iPer] + " )" )
    hRcp.GetXaxis().SetTitle( "p_T [GeV]" )
    hRcp.SetBinContent( 1, 0 )
    hRcp.SetBinContent( 2, 0 )
    hRcp.SetBinContent( 3, 0 )
    
    if 'trunc' in kwargs :
        for bin in range( 1, hRcp.GetNbinsX()+1 ) :
            if bin >= kwargs[ 'trunc' ] :
                hRcp.SetBinContent( bin, 0 )
                hRcp.SetBinError( bin, 0 )
    
    hRcp.Draw()
    if 'color' in kwargs :
        hRcp.SetMarkerColor( kwargs[ 'color' ] )
    hRcp.GetYaxis().SetRangeUser( 0.1, 13 )
    